### 13C chemical shifts assignment tool
Input of structure and chemical shifts  

! It must match the number of C atoms with the number of shifts in the version!  

! If there are chemically equivalent nuclei, the shift must be entered once for each nucleus.  

It is sorted by size!

In the current version, no checks of the input are performed.

The chemical shifts are entered consecutively, separated by spaces. Both the English number format (128.5) and the German number format (128,5) are supported. The entry of the SMILES code and the list of shifts is completed by pressing Enter. Then, proceed with the next cell in the execution.

In [5]:
######################################################################
# In the first code cell, the data is read into the pandas DataFrame.
# Then it doesn't need to be done again every time.

import pandas as pd
from rdkit import *
from rdkit.Chem import AllChem, Draw
from IPython.display import display, Image, HTML
import os, sys
import tempfile
from dcode import geometry, tools
from dcode.geometry import *
from dcode.tools import *
from dcode.calcshift import *


codefile="codes/v3_update_10_06_2025.csv"

# reading data for calculation
df = pd.read_csv(codefile)




In [48]:
#########################################################################
#                                                                       #
#         I N P U T                                                     #
#                                                                       #
#########################################################################

########  SMILES Code of the compound of interest #################

#Example
#smiles=r"[H]O[C@]1([H])[C@@]2([H])C([H])([H])C(=O)[C@]([H])(C2([H])[H])[C@]1([H])O[H]"


##########################################################################
#                                                                        #
#     List of experimental chemical shifts                               #
#                                                                        #
#        one 13C chemical shift for each C-Atomm                         #
#                                                                        #
##########################################################################
#Beispiel
#ExpShift=r"71.6;44.7;43.9;223.9;61.1;34.0;75.2"

smiles=input("SMILES-String:")

print(smiles)
print()
print("chemical shifts (space separated)")
print()

InputShift=input("Enter 13C chemical shifts:")


ExpShift=InputShift.replace(',', '.').replace(' ',';')

print(ExpShift)
print
print("continue with next cell")



SMILES-String:[H]c1oc2nc([H])c3c([H])c([N+](=O)[O-])oc3c2c1[H]
[H]c1oc2nc([H])c3c([H])c([N+](=O)[O-])oc3c2c1[H]

chemical shifts (space separated)

Enter 13C chemical shifts:145,50 162,90 140,80 119,30 106,60 153,00 151,80 105,10 102,90
145.50;162.90;140.80;119.30;106.60;153.00;151.80;105.10;102.90
continue with next cell


In [49]:
####  exp. Verschiebungen in Dataframe umformen ####

# String in eine Liste umwandeln
werte_liste = ExpShift.split(';')

# Liste in ein DataFrame umwandeln
expdata = pd.DataFrame(werte_liste, columns=["Shift [ppm]"])

# Optional: Werte in den richtigen Datentyp umwandeln (float)
expdata["Shift [ppm]"] = expdata["Shift [ppm]"].astype(float)

# DataFrame anzeigen für Debug
#print(expdata)


#### Verschiebungsberechnung Teil 1 ###

groesse=(450,450)   # definiert die Größe der Abbildung
current_dir = os.getcwd()

mol = Chem.MolFromSmiles(smiles) #Molecule from SMILES

molH=mol

if not Hda(mol): 
    molH=Chem.AddHs(mol)

molID=AllChem.EmbedMolecule(molH,randomSeed=435542)  # generate 3D coordinats 

# look for success

if (molID<0):
   sys.exit('Error in 3D coordinate generation. Program will exit.')

## optimize molecule
molOpt= ffoptimize(molH)

molCoded=DCodeName(molOpt)
## build distances
#molcoded is a molecule object with codes as an atomic property.
#Function call to store the code as an atomic property.
#As a second parameter, sort= Integer can be passed for the OZ of the atom type of interest.
#The default is 6 for carbon.
mol_mit_Code=DCodeMol(molCoded)
## DCode is now an atomic propertie
    

## build molecule
show_atom_number(molOpt, 'atomNote') #create numbering
mol_ohne_H = Chem.RemoveHs(molOpt)
AllChem.Compute2DCoords(mol_ohne_H)
img = Draw.MolToImage(mol_ohne_H, size=groesse)  # change size
## using tempfile library to create a temporary image to avoid problems updating the display
## molbild is a temporary name for the picture


with tempfile.NamedTemporaryFile(dir=os.getcwd(), suffix='.png', delete=False) as temp_file:
   molbild=temp_file.name
   img.save(molbild)  # save image

print("continue with next cell")

continue with next cell


In [50]:
# Beispiel für table_data (ersetze dies durch deine tatsächlichen Daten)
table_data = []

# Beispiel für den DataFrame, der Daten sammeln würde
for atom in mol_mit_Code.GetAtoms():
    if atom.HasProp('DCode'):
        codestring = atom.GetProp('DCode')
        index = atom.GetIdx() + 1
        verschiebung, trefferzahl, kuerzungen = calcshift(df, codestring)
        
        # Daten in die Tabelle einfügen
        table_data.append({
            "Atom": index,
            "Shift [ppm]": f"{verschiebung:.2f}",
            "Hits": trefferzahl,
            "Shorts": kuerzungen - 1 if (kuerzungen - 1) != 0 else 0
        })

# Erstelle den Pandas DataFrame
df_table = pd.DataFrame(table_data)

# Erstelle eine Liste der verfügbaren Werte aus expdata
expdata_values = expdata["Shift [ppm]"].tolist()

# Sortiere die Werte
df_table['Shift [ppm]'] = df_table['Shift [ppm]'].astype(float)  # Stelle sicher, dass die Werte als float sind
df_table_sorted = df_table.sort_values(by='Shift [ppm]').reset_index(drop=True)
expdata_sorted = sorted(expdata_values)

# Weise die Werte der Größe nach zu
min_length = min(len(df_table_sorted), len(expdata_sorted))
df_table_sorted['Assigned Shift [ppm]'] = expdata_sorted[:min_length]

# Optional: Wenn df_table mehr Zeilen hat als expdata, setze die restlichen Werte auf NaN oder einen anderen Wert
if len(df_table_sorted) > len(expdata_sorted):
    df_table_sorted['Assigned Shift [ppm]'].fillna('N/A', inplace=True)
    
# Sortiere df_table_sorted nach der Spalte 'Atom' aufsteigend
df_table_sorted = df_table_sorted.sort_values(by='Atom').reset_index(drop=True)

# DataFrame anzeigen
#print(df_table_sorted)


molbild_filename = os.path.basename(molbild)

# HTML-Template 
html_content = f"""
<div style="display: flex; align-items: flex-start;">
    <!-- Bild -->
    <div style="margin-right: 20px;">
        <img src="{molbild_filename}" alt="Molbild" style="width:450px;height:auto;">
    </div>
    
    <!-- Tabelle -->
    <div>
        {df_table_sorted.to_html(escape=False, index=False)}
    </div>
</div>
"""

# show HTML
display(HTML(html_content))


Atom,Shift [ppm],Hits,Shorts,Assigned Shift [ppm]
1,143.9,1,0,145.5
3,161.0,1,1,162.9
5,138.8,3,4,140.8
6,120.9,7,8,119.3
7,105.1,1,4,106.6
8,153.2,1,6,151.8
13,153.8,3,6,153.0
14,104.8,1,3,105.1
15,102.5,1,3,102.9
